In [1]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
neu_data = all_data[all_data["labels"] == "NE"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
neu_train = neu_data.iloc[0:round(ratio*neu_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
neu_test = neu_data.iloc[round(ratio*neu_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [2]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_neu = set(' '.join(neu_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario neutro: " + str(len(vocab_neu)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2692
Dimensão vocabulario neutro: 2153
Dimensão vocabulario positivo: 2330


### Removendo somente a interseção central

In [3]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos.intersection(vocab_neu))
vocab_neg_excl = vocab_neg.difference(intersect)
vocab_neu_excl = vocab_neu.difference(intersect)
vocab_pos_excl = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [8]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.537735849057
Fold  1  - Acuracia:  0.490384615385
Fold  2  - Acuracia:  0.528846153846
Fold  3  - Acuracia:  0.5
Fold  4  - Acuracia:  0.509615384615
Fold  5  - Acuracia:  0.576923076923
Fold  6  - Acuracia:  0.596153846154
Fold  7  - Acuracia:  0.586538461538
Fold  8  - Acuracia:  0.586538461538
Fold  9  - Acuracia:  0.548076923077

Accuracia media:  0.546081277213
Desvio padrão:  0.0369568506763


SVM
Cross Validation:
Fold  0  - Acuracia:  0.584905660377
Fold  1  - Acuracia:  0.528846153846
Fold  2  - Acuracia:  0.442307692308
Fold  3  - Acuracia:  0.480769230769
Fold  4  - Acuracia:  0.576923076923
Fold  5  - Acuracia:  0.548076923077
Fold  6  - Acuracia:  0.576923076923
Fold  7  - Acuracia:  0.557692307692
Fold  8  - Acuracia:  0.586538461538
Fold  9  - Acuracia:  0.528846153846

Accuracia media:  0.54118287373
Desvio padrão:  0.0453587081655

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.556603773585
Fold  1  - Acuracia: 

In [9]:
def get_ngram_lexicon(ngram_range):
    # Avaliando a incidência dos bigramas
    cv = CountVectorizer(ngram_range=ngram_range, stop_words = stopwords_pt)

    neg_counts = cv.fit_transform(neg_train["texts"])
    neg_ngrams = set(cv.vocabulary_)

    pos_counts = cv.fit_transform(pos_train["texts"])
    pos_ngrams = set(cv.vocabulary_)

    neu_counts = cv.fit_transform(neu_train["texts"])
    neu_ngrams = set(cv.vocabulary_)

    # Encontrando as interseções
    central_intersection = pos_ngrams.intersection(neg_ngrams).intersection(neu_ngrams)
    pos_neg_intersection = pos_ngrams.intersection(neg_ngrams)
    pos_neu_intersection = pos_ngrams.intersection(neu_ngrams)
    neu_neg_intersection = neu_ngrams.intersection(neg_ngrams)

    total_intersection = central_intersection.union(pos_neg_intersection).union(pos_neu_intersection).union(neu_neg_intersection)

    #Removendo as interseções do vocabulario
    lexicon_ngrams = neg_ngrams.union(pos_ngrams).union(neu_ngrams).difference(total_intersection)

    print("Dimensionalidade: ", len(lexicon_ngrams))
    return lexicon_ngrams

# neg_counts = cv.fit_transform(neg_train["texts"])
# neg_ngrams = cv.vocabulary_

# pos_counts = cv.fit_transform(pos_train["texts"])
# pos_ngrams = cv.vocabulary_

# neu_counts = cv.fit_transform(neu_train["texts"])
# neu_ngrams = cv.vocabulary_

# spo = np.sum(pos_counts.toarray(), axis=0)
# sng = np.sum(neg_counts.toarray(), axis=0)
# sne = np.sum(neu_counts.toarray(), axis=0)

# pos_ngrams  = {ngram: spo[index] for ngram, index in pos_ngrams.items()}
# neg_ngrams  = {ngram: sng[index] for ngram, index in neg_ngrams.items()}
# neu_ngrams  = {ngram: sne[index] for ngram, index in neu_ngrams.items()}

# pos_excl_ngrams = {key:value for key, value in pos_ngrams.items() if key not in neg_ngrams and key not in neu_ngrams}
# neg_excl_ngrams = {key:value for key, value in neg_ngrams.items() if key not in pos_ngrams and key not in neu_ngrams}
# neu_excl_ngrams = {key:value for key, value in neu_ngrams.items() if key not in pos_ngrams and key not in neg_ngrams}

# lexicon_ngrams = list(pos_excl_ngrams.keys()) + list(neg_excl_ngrams.keys()) + list(neu_excl_ngrams)

In [14]:
from pandas import Series
s = Series(neg_excl_ngrams)
s.nlargest(15)

edison               12
edison lobão         12
sabesp               11
calheiros pmdb        8
lobão minas           8
ministro edison       8
presidentes           8
beneficiários         7
collor                7
mário                 7
mário negromonte      7
acabou                6
governadora           6
jânio                 6
líderes congresso     6
dtype: int64

### Somente com Unigrams

In [12]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(1,1))

print("UNIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,1), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")
print("UNIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,1), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  2913
UNIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.622949927431
Desvio padrão:  0.0441834796332

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.643033381713
Desvio padrão:  0.030991717085

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.58448838897
Desvio padrão:  0.0442820742617



UNIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.62198838897
Desvio padrão:  0.0438129682017

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.651687227866
Desvio padrão:  0.0308633974152

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.583508708273
Desvio padrão:  0.0492484030048


### Somente com Bigrams

In [14]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(2,2))

print("BIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")

print("BIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  13332
BIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.591055878084
Desvio padrão:  0.0485226924291

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.645881712627
Desvio padrão:  0.0341021374824

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.48084179971
Desvio padrão:  0.0294336393163



BIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.590076197388
Desvio padrão:  0.0531563928766

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.645863570392
Desvio padrão:  0.0411555314181

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.48084179971
Desvio padrão:  0.0335442224988


### Bigrams + Unigrams

In [15]:
lexicon_ngrams = get_ngram_lexicon(ngram_range=(1,2))

print("UNIGRAMS + BIGRAMS FREQUENCY**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams))
                    ])
evaluate(all_data, features, n_folds= 10)

print("\n\n")

print("UNIGRAMS + BIGRAMS PRESENCE**************************************")
features = FeatureUnion([
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams, binary= True))
                    ])
evaluate(all_data, features, n_folds= 10)

Dimensionalidade:  16245
UNIGRAMS + BIGRAMS FREQUENCY**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.658291001451
Desvio padrão:  0.0243226756047

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.702539912917
Desvio padrão:  0.0333809435451

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.568232946299
Desvio padrão:  0.0326479035624



UNIGRAMS + BIGRAMS PRESENCE**************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.663098693759
Desvio padrão:  0.0285017920489

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.703465166909
Desvio padrão:  0.0231368985063

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.563425253991
Desvio padrão:  0.0334123825789


### Usando LSA e MaxEnt

In [17]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion

pca = PCA()
# pca = TruncatedSVD(n_components=500)
cv  = CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_ngrams)

t_data = cv.fit_transform(all_data["texts"])
t_data = pca.fit_transform(t_data.toarray())

labels = np.array([])
for label in all_data["labels"]:
    if label == 'PO':
        labels = np.append(labels, [1])
    elif label == "NG":
        labels = np.append(labels, [-1])
    elif label == "NE":
        labels = np.append(labels, [0])

labels.shape

classifier = LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False)
# classifier = MultinomialNB()

print("Cross Validation:")
accuracy_average = np.array([])
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
for index, (train, test) in enumerate(sKFold.split(t_data, labels)):
    # Treinando um modelo Naive Bayes
    train_data = t_data[train]
    test_data = t_data[test]

    classifier.fit(train_data, labels[train])
    predictions = classifier.predict(test_data)

    accuracy = accuracy_score(labels[test], predictions, normalize=True)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    #print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Accuracia media:  0.420301161103
Desvio padrão:  0.0325551831623
